In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

#fname = "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/127C2975-1B1C-A046-AABF-62B77E757A86.root"
#fname = "file:/opendata_cms/DoubleMuon/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD/127C2975-1B1C-A046-AABF-62B77E757A86.root"
fname = "file:///code/data/EEB2FE3F-7CF3-BF4A-9F70-3F89FACE698E.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DoubleMuon"},
).events()

print("Number of events = ", ak.num(events.Muon, axis=0).compute())

/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cr

Number of events =  2167324


In [2]:
print("Number of muons = ", ak.count(events.Muon.mass, axis=None).compute())

Number of muons =  4749697


In [3]:
selected_muons = events.Muon[(events.Muon.isGlobal == 1)]
print("Number of selected muons after isGlobal==1: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after isGlobal==1:  4126670


In [4]:
selected_muons = selected_muons[(selected_muons.nStations > 0)]
print("Number of selected muons after nStations>0: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nStations>0:  3960328


In [5]:
selected_muons = selected_muons[(selected_muons.nTrackerLayers > 5)]
print("Number of selected muons after nTrackerLayers>5: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nTrackerLayers>5:  3939447


In [6]:
selected_muons = selected_muons[(selected_muons.dxy < 0.3) & (selected_muons.dz < 20.0)]
print("Number of selected muons after PV impact param sel: ", ak.count(selected_muons.mass, axis=None).compute())       

Number of selected muons after PV impact param sel:  3924428


In [7]:
selected_muons = selected_muons[(selected_muons.pt > 3.0) & (abs(selected_muons.eta) < 2.4)]
print("Number of selected muons after pt,eta sel: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after pt,eta sel:  3915277


In [8]:
selected_muons = selected_muons[(selected_muons.pfRelIso03_chg < 0.5)]
print("Number of selected muons after ptcone30 isolation: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after ptcone30 isolation:  2656161


In [9]:
selectede_muons = selected_muons[ak.num(selected_muons) > 0]
print("Number of selected events with num(muons)>0: ", ak.num(selectede_muons.mass, axis=0).compute())
print("Corresponding number of selected muons: ", ak.count(selectede_muons.mass, axis=None).compute())

Number of selected events with num(muons)>0:  1605413
Corresponding number of selected muons:  2656161


In [10]:
#selected_muons_c = selected_muons.compute()
pass

In [11]:
fourmuon = ak.combinations(selected_muons, 4, fields=['lep1','lep2','lep3','lep4'])

In [12]:
print(type(fourmuon))

<class 'dask_awkward.lib.core.Array'>


In [13]:
print("Number of 4-muon combination: ", ak.count(fourmuon.lep1.mass, axis=None).compute())

Number of 4-muon combination:  5361


In [14]:
fourmuon_c = fourmuon.compute()

In [15]:
print("Number of 4-muon (computed) combination: ", ak.count(fourmuon_c.lep1.mass, axis=None))

Number of 4-muon (computed) combination:  5361


In [16]:
# should be the same as above
print("Number of 4-muon (computed) combination: ", ak.count(fourmuon_c.lep4.pt, axis=None))

Number of 4-muon (computed) combination:  5361


In [17]:
print(fourmuon_c.lep1.charge[ak.num(fourmuon_c)>1] + fourmuon_c.lep2.charge[ak.num(fourmuon_c)>1])

[[0, 0, 0, 0, 2], [0, 0, 0, 0, -2], ..., [2, 2, 2, 0, 0], [0, 0, 0, 0, -2]]


In [18]:
#fourmuon_c = fourmuon_c[((fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0) and
#                         (fourmuon_c.lep3.charge + fourmuon_c.lep4.charge == 0))]
#fourmuon_c = fourmuon_c[(fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0)]
fourmuon_c = fourmuon_c[(fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0) &
                        (fourmuon_c.lep3.charge + fourmuon_c.lep4.charge == 0)]

In [19]:
xxx = fourmuon_c.lep1.charge + fourmuon_c.lep2.charge == 0

In [20]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', '[', end=' ')
    for quad in qquad:
        print(quad.lep1.charge+quad.lep2.charge==0, quad.lep3.charge+quad.lep4.charge==0, end=' ')
    print(']')
    i = i+1
    if i>100:
        break

1919 : [ True True ]
2847 : [ True True ]
4349 : [ True True ]
4698 : [ True True ]
6126 : [ True True ]
6528 : [ True True ]
8683 : [ True True ]
9151 : [ True True ]
9936 : [ True True ]
11502 : [ True True ]
12688 : [ True True ]
13494 : [ True True ]
16488 : [ True True ]
16586 : [ True True ]
17331 : [ True True ]
17408 : [ True True ]
18051 : [ True True ]
18115 : [ True True ]
19949 : [ True True ]
23194 : [ True True True True True True ]
24563 : [ True True ]
26306 : [ True True ]
27613 : [ True True ]
28426 : [ True True ]
28618 : [ True True True True ]
30842 : [ True True ]
35469 : [ True True ]
36226 : [ True True ]
37456 : [ True True True True ]
37539 : [ True True ]
38518 : [ True True ]
38721 : [ True True ]
39421 : [ True True ]
40769 : [ True True ]
43902 : [ True True ]
45254 : [ True True ]
47836 : [ True True ]
49011 : [ True True ]
50005 : [ True True ]
50574 : [ True True ]
52864 : [ True True True True ]
53268 : [ True True ]
55539 : [ True True ]
56180 : [ Tru

In [21]:
#countset = set()
#for j, qquad in enumerate(fourmuon_c):
#    countset.add(len(qquad))
#print(countset)

In [22]:
#counthist = {}
#for qquad in fourmuon_c:
#    j = len(qquad)
#    if j in counthist:
#        counthist[j] += 1
#    else:
#        counthist[j] = 1
#print(counthist)

In [23]:
#i = 0
#for j, qquad in enumerate(fourmuon_c):
#    if len(qquad) == 0:
#        continue
#    print(j, ':', '[', end=' ')
#    for quad in qquad:
#        print(type(quad), end=' ')
#        print(type(quad.lep1), end=' ')
#        print(type(quad.lep1.pt), end=' ')
#        print(quad.lep1.pt, quad.lep2.pt, quad.lep3.pt, quad.lep4.pt, end=' ')
#    print(']')
#    i = i+1
#    if i>100:
#        break

In [24]:
print(fourmuon_c[1919].lep1.pt)

[11.5]


In [25]:
print(type(fourmuon_c[1919].lep1.pt))

<class 'awkward.highlevel.Array'>


In [26]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.lep1.pt, qquad.lep2.pt, qquad.lep3.pt, qquad.lep4.pt)
    i += 1
    if i>100:
        break

1919 : [11.5] [6.81] [4.18] [3.48]
2847 : [7.22] [5.21] [4.07] [3.52]
4349 : [12.8] [4.47] [4.4] [3.66]
4698 : [56.8] [45.9] [4.48] [3.19]
6126 : [15.8] [13] [6.61] [4.72]
6528 : [16.2] [13.1] [3.83] [3.1]
8683 : [5.49] [4.3] [3.66] [3.56]
9151 : [11.1] [6.46] [6.01] [5.69]
9936 : [9.83] [7.87] [6.27] [3.11]
11502 : [12.8] [7.68] [4.23] [3.82]
12688 : [5.78] [4.68] [3.66] [3.1]
13494 : [7.05] [5.55] [4.96] [4.75]
16488 : [15.7] [7.42] [5.04] [3.55]
16586 : [13.3] [5.95] [4.65] [4.57]
17331 : [9.94] [6.86] [5.44] [4.01]
17408 : [33.4] [17.8] [10.8] [3.62]
18051 : [24.2] [9.64] [4.58] [3.43]
18115 : [8.06] [5.77] [3.15] [3.09]
19949 : [13.4] [8.83] [5.91] [3.89]
23194 : [46.9, 46.9, 46.9] [35.5, 35.5, 8.61] [8.61, 5.83, 5.83] [4.56, 4.56, 4.56]
24563 : [9.35] [6.38] [5.51] [4.67]
26306 : [67] [48.5] [7.12] [3.93]
27613 : [5.81] [4.83] [4.77] [3.94]
28426 : [9] [6.74] [3.36] [3.05]
28618 : [46.4, 46.4] [46, 46] [4.88, 4.88] [3.65, 3.32]
30842 : [25] [7.78] [7.74] [3.47]
35469 : [10.1] [8.

In [27]:
fourmuon_c["pair1"] = fourmuon_c.lep1 + fourmuon_c.lep2
fourmuon_c["pair2"] = fourmuon_c.lep3 + fourmuon_c.lep4

In [28]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.pair1.mass, qquad.pair2.mass)
    i += 1
    if i>200:
        break

1919 : [2.76] [23.3]
2847 : [15.9] [6.14]
4349 : [18.9] [3.13]
4698 : [102] [5.64]
6126 : [2.33] [9.2]
6528 : [3.11] [10.6]
8683 : [1.31] [7.83]
9151 : [2.48] [2.02]
9936 : [3.1] [7.1]
11502 : [16.9] [1.36]
12688 : [5.47] [5.21]
13494 : [3.13] [3.07]
16488 : [17.7] [13.8]
16586 : [20] [10.3]
17331 : [3.1] [1.19]
17408 : [66.3] [1.71]
18051 : [1.5] [1.22]
18115 : [15.4] [16.5]
19949 : [2.83] [9.54]
23194 : [82.2, 82.2, 7.35] [12.2, 14.5, 14.5]
24563 : [13.5] [3.11]
26306 : [87.6] [2.7]
27613 : [1.61] [13.7]
28426 : [9.5] [5.67]
28618 : [100, 100] [12.4, 6.32]
30842 : [2.33] [2.74]
35469 : [39] [6.9]
36226 : [22.4] [8.7]
37456 : [11.8, 11.8] [10.2, 2.48]
37539 : [2] [18.3]
38518 : [16.5] [4.42]
38721 : [2.99] [2.49]
39421 : [54.1] [4.73]
40769 : [22.6] [11.5]
43902 : [12] [2.51]
45254 : [96] [18.5]
47836 : [214] [0.694]
49011 : [3.09] [2.24]
50005 : [1.6] [9.52]
50574 : [82.7] [10.3]
52864 : [15.5, 15.2] [9.89, 9.89]
53268 : [0.517] [16.8]
55539 : [58.3] [3.45]
56180 : [32.2] [6.43]
5655

In [29]:
i = 0
for j, qquad in enumerate(fourmuon_c):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad.pair1.mass[0], qquad.pair2.mass[0])
    i += 1
    if i>30:
        break

1919 : 2.7637265 23.300982
2847 : 15.935995 6.1425953
4349 : 18.853579 3.1254394
4698 : 102.11299 5.6351604
6126 : 2.3279676 9.196075
6528 : 3.1055253 10.61169
8683 : 1.3084247 7.833044
9151 : 2.4824333 2.0158143
9936 : 3.0991855 7.1049223
11502 : 16.919518 1.3569145
12688 : 5.46732 5.2098713
13494 : 3.1341467 3.0732284
16488 : 17.749794 13.763896
16586 : 19.962112 10.2961445
17331 : 3.100352 1.1889254
17408 : 66.254776 1.7075183
18051 : 1.5011389 1.220142
18115 : 15.38485 16.541796
19949 : 2.8306704 9.537453
23194 : 82.24434 12.204126
24563 : 13.544095 3.1148076
26306 : 87.5514 2.6966653
27613 : 1.6136926 13.684553
28426 : 9.502197 5.6742525
28618 : 99.95341 12.35397
30842 : 2.3288066 2.743645
35469 : 39.043915 6.898674
36226 : 22.4384 8.698746
37456 : 11.801856 10.175123
37539 : 2.002196 18.327385
38518 : 16.517954 4.417629


In [30]:
i = 0
fourmuon_t1 = abs(fourmuon_c.pair1.mass - 3.096916)
for j, qquad in enumerate(fourmuon_t1):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad, qquad)
    i += 1
    if i>30:
        break

1919 : [0.333] [0.333]
2847 : [12.8] [12.8]
4349 : [15.8] [15.8]
4698 : [99] [99]
6126 : [0.769] [0.769]
6528 : [0.00861] [0.00861]
8683 : [1.79] [1.79]
9151 : [0.614] [0.614]
9936 : [0.00227] [0.00227]
11502 : [13.8] [13.8]
12688 : [2.37] [2.37]
13494 : [0.0372] [0.0372]
16488 : [14.7] [14.7]
16586 : [16.9] [16.9]
17331 : [0.00344] [0.00344]
17408 : [63.2] [63.2]
18051 : [1.6] [1.6]
18115 : [12.3] [12.3]
19949 : [0.266] [0.266]
23194 : [79.1, 79.1, 4.25] [79.1, 79.1, 4.25]
24563 : [10.4] [10.4]
26306 : [84.5] [84.5]
27613 : [1.48] [1.48]
28426 : [6.41] [6.41]
28618 : [96.9, 96.9] [96.9, 96.9]
30842 : [0.768] [0.768]
35469 : [35.9] [35.9]
36226 : [19.3] [19.3]
37456 : [8.7, 8.7] [8.7, 8.7]
37539 : [1.09] [1.09]
38518 : [13.4] [13.4]


In [31]:
i = 0
fourmuon_t1 = abs(fourmuon_c.pair1.mass - 9.46030)
for j, qquad in enumerate(fourmuon_t1):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad, qquad)
    i += 1
    if i>30:
        break

1919 : [6.7] [6.7]
2847 : [6.48] [6.48]
4349 : [9.39] [9.39]
4698 : [92.7] [92.7]
6126 : [7.13] [7.13]
6528 : [6.35] [6.35]
8683 : [8.15] [8.15]
9151 : [6.98] [6.98]
9936 : [6.36] [6.36]
11502 : [7.46] [7.46]
12688 : [3.99] [3.99]
13494 : [6.33] [6.33]
16488 : [8.29] [8.29]
16586 : [10.5] [10.5]
17331 : [6.36] [6.36]
17408 : [56.8] [56.8]
18051 : [7.96] [7.96]
18115 : [5.92] [5.92]
19949 : [6.63] [6.63]
23194 : [72.8, 72.8, 2.11] [72.8, 72.8, 2.11]
24563 : [4.08] [4.08]
26306 : [78.1] [78.1]
27613 : [7.85] [7.85]
28426 : [0.0419] [0.0419]
28618 : [90.5, 90.5] [90.5, 90.5]
30842 : [7.13] [7.13]
35469 : [29.6] [29.6]
36226 : [13] [13]
37456 : [2.34, 2.34] [2.34, 2.34]
37539 : [7.46] [7.46]
38518 : [7.06] [7.06]


In [ ]:
i = 0
fourmuon_t1 = ak.min(ak.Array([abs(fourmuon_c.pair1.mass - 3.096916), abs(fourmuon_c.pair1.mass - 9.46030)]), axis=0)
for j, qquad in enumerate(fourmuon_t1):
    if len(qquad) == 0:
        continue
    print(j, ':', qquad, qquad)
    i += 1
    if i>30:
        break

In [ ]:
jpsimass = 3.096916
upsilonmass = 9.46030
bestmass = ak.singletons(
               ak.argmin(
                   ak.min(
                       ak.Array([abs(fourmuon_c.pair1.mass - jpsimass), abs(fourmuon_c.pair1.mass - upsilonmass)]), axis=0
                   ) +
                   ak.min(
                       ak.Array([abs(fourmuon_c.pair2.mass - jpsimass), abs(fourmuon_c.pair2.mass - upsilonmass)]), axis=0
                   ),
                   axis=1
               )
)